In [17]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

# 한글 폰트 설정 및 음수 부호 깨짐 방지
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 'data' 폴더가 없으면 생성
data_folder = os.path.join('app_manager', 'data')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# headers 설정
headers = {"User-Agent": "Mozilla/5.0"}

# 종목 코드와 기업명 매핑
stock_info = {
    '현대차': '005380',
    '기아': '000270',
    '에코프로비엠': '247540',
    'LG화학': '051910',
    '금양': '001570',
    'SK이노베이션': '096770',
    'LG 에너지솔루션': '373220',
    '삼성SDI': '006400',
    '엘앤에프': '066970',
    '포스코퓨처엠': '003670',
    '포스코DX': '022100',
    'SK 네트웍스': '001740',
    '솔루엠': '248070',
    '롯데이노베이트': '286940'
}

def get_data(stock_code, pages=1):
    data = []

    for page in range(1, pages + 1):
        base_url = f'https://finance.naver.com/item/sise_day.nhn?code={stock_code}'
        params = {
            "code": stock_code,
            "page": page
        }
        try:
            res = requests.get(base_url, headers=headers, params=params)
            res.raise_for_status()
            res.encoding = 'euc-kr'
            soup = BeautifulSoup(res.text, "html.parser")

            table = soup.find("table", {"class": "type2"})
            if not table:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                continue

            rows = table.find_all("tr")
            for row in rows:
                cols = row.find_all("td")
                if len(cols) > 1:  # 공백 행 제외
                    try:
                        date = cols[0].get_text(strip=True)
                        closing_price = cols[1].get_text(strip=True).replace(',', '')
                        previous_close = cols[2].get_text(strip=True)
                        opening_price = cols[3].get_text(strip=True).replace(',', '')
                        high_price = cols[4].get_text(strip=True).replace(',', '')
                        low_price = cols[5].get_text(strip=True).replace(',', '')
                        volume = cols[6].get_text(strip=True).replace(',', '')

                        # 전일비 처리
                        if "상승" in previous_close:
                            previous_close_value ="{:+}".format(int(''.join(filter(str.isdigit, previous_close))))
                        elif "하락" in previous_close:
                            previous_close_value = -int(''.join(filter(str.isdigit, previous_close)))
                        else:
                            previous_close_value = 0

                        parsed_date = pd.to_datetime(date, format='%Y.%m.%d')

                        data.append({
                            '날짜': parsed_date,
                            '전일비': previous_close_value,
                            '시가': int(opening_price) if opening_price.isdigit() else None,
                            '고가': int(high_price) if high_price.isdigit() else None,
                            '저가': int(low_price) if low_price.isdigit() else None,
                            '종가': int(closing_price) if closing_price.isdigit() else None,
                            '거래량': int(volume) if volume.isdigit() else None
                        })

                    except ValueError as ve:
                        print(f"데이터 변환 오류: {ve}")

            time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"HTTP 요청 오류: {e}")

    df = pd.DataFrame(data)
    df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')
    df.dropna(subset=['날짜'], inplace=True)


    df.sort_values('날짜', inplace=True, ascending=False)
    df.reset_index(drop=True, inplace=True)

    return df

# 각 종목의 데이터를 가져와 기업별로 CSV 파일로 저장
for company_name, stock_code in stock_info.items():
    print(f"{company_name} ({stock_code})의 데이터를 가져오는 중...")

    # 종목의 데이터를 가져옴
    stock_data = get_data(stock_code=stock_code)

    # 기업명을 파일 이름에 포함하여 CSV 저장
    csv_filename = os.path.join(data_folder, f'{company_name}_full_daily_prices.csv')
    stock_data.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"{csv_filename} 파일이 저장되었습니다.")




현대차 (005380)의 데이터를 가져오는 중...
app_manager\data\현대차_full_daily_prices.csv 파일이 저장되었습니다.
기아 (000270)의 데이터를 가져오는 중...
app_manager\data\기아_full_daily_prices.csv 파일이 저장되었습니다.
에코프로비엠 (247540)의 데이터를 가져오는 중...
app_manager\data\에코프로비엠_full_daily_prices.csv 파일이 저장되었습니다.
LG화학 (051910)의 데이터를 가져오는 중...
app_manager\data\LG화학_full_daily_prices.csv 파일이 저장되었습니다.
금양 (001570)의 데이터를 가져오는 중...
app_manager\data\금양_full_daily_prices.csv 파일이 저장되었습니다.
SK이노베이션 (096770)의 데이터를 가져오는 중...
app_manager\data\SK이노베이션_full_daily_prices.csv 파일이 저장되었습니다.
LG 에너지솔루션 (373220)의 데이터를 가져오는 중...
app_manager\data\LG 에너지솔루션_full_daily_prices.csv 파일이 저장되었습니다.
삼성SDI (006400)의 데이터를 가져오는 중...
app_manager\data\삼성SDI_full_daily_prices.csv 파일이 저장되었습니다.
엘앤에프 (066970)의 데이터를 가져오는 중...
app_manager\data\엘앤에프_full_daily_prices.csv 파일이 저장되었습니다.
포스코퓨처엠 (003670)의 데이터를 가져오는 중...
app_manager\data\포스코퓨처엠_full_daily_prices.csv 파일이 저장되었습니다.
포스코DX (022100)의 데이터를 가져오는 중...
app_manager\data\포스코DX_full_daily_prices.csv 파일이 저장되었습니다.
SK 네트웍스 (001740)의 데이터를 가져